Using TensorFlow and the RNEM model on the MovieLens dataset to recommend a movie to a user based on their past ratings.

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf

Load the MovieLens dataset

In [ ]:

movies_df = pd.read_csv("https://cdn.sstatic.net/datasets/movielens/ml-latest-small/movies.csv")
ratings_df = pd.read_csv("https://cdn.sstatic.net/datasets/movielens/ml-latest-small/ratings.csv")

Filter the dataset to only include movies and users with a certain number of ratings

In [ ]:

min_movie_ratings = 50
min_user_ratings = 50
movie_counts = ratings_df["movieId"].value_counts()
user_counts = ratings_df["userId"].value_counts()
valid_movies = movie_counts[movie_counts >= min_movie_ratings].index
valid_users = user_counts[user_counts >= min_user_ratings].index
ratings_df = ratings_df[(ratings_df["movieId"].isin(valid_movies)) & (ratings_df["userId"].isin(valid_users))]

Map movie and user IDs to contiguous integers

In [ ]:

unique_movies = ratings_df["movieId"].unique()
unique_users = ratings_df["userId"].unique()
movie_id_map = dict(zip(unique_movies, range(len(unique_movies))))
user_id_map = dict(zip(unique_users, range(len(unique_users))))
ratings_df["movieId"] = ratings_df["movieId"].map(movie_id_map)
ratings_df["userId"] = ratings_df["userId"].map(user_id_map)

Split the dataset into training and testing sets

In [ ]:

train_ratio = 0.8
train_mask = np.random.rand(len(ratings_df)) < train_ratio
train_ratings = ratings_df[train_mask]
test_ratings = ratings_df[~train_mask]

Compute the number of entities and relations

In [ ]:

num_movies = len(unique_movies)
num_users = len(unique_users)
num_ratings = len(ratings_df)
num_relations = 2
num_entities = num_movies + num_users

Define the RNEM model architecture

In [ ]:

def RNEM(num_relations, num_entities, hidden_size):
    # Define placeholders for input data
    relations = tf.placeholder(tf.float32, [None, num_relations])
    entities = tf.placeholder(tf.float32, [None, num_entities])

    # Define a fully connected layer to encode the relations
    relations_encoding = tf.layers.dense(relations, hidden_size, activation=tf.nn.relu)

    # Define a fully connected layer to encode the entities
    entities_encoding = tf.layers.dense(entities, hidden_size, activation=tf.nn.relu)

    # Define a matrix multiplication layer to compute the likelihoods of each relation
    logits = tf.matmul(relations_encoding, tf.transpose(entities_encoding))

    # Define a sigmoid activation function to compute the probabilities
    probabilities = tf.nn.sigmoid(logits)

    # Define a placeholder for the ground truth data
    ground_truth = tf.placeholder(tf.float32, [None, num_relations])

    # Define a binary cross-entropy loss function
    loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=ground_truth, logits=logits))

    # Define an optimizer to minimize the loss
    optimizer = tf.train.AdamOptimizer(learning_rate=0.01).minimize(loss)

    # Return the model components
    return relations, entities, probabilities, ground_truth, loss, optimizer


Instantiate the RNEM model

In [ ]:

hidden_size = 50
relations, entities, probabilities, ground_truth, loss, optimizer = RNEM(num_relations, num_entities, hidden_size)

Initialize a TensorFlow session and run the optimizer

In [ ]:

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(100):
        # Sample a batch of training data
        batch_size = 1000
        indices = np.random.choice(len(train_ratings), batch_size)
        batch_relations = np.zeros((batch_size, num_relations))
        batch_entities = np.zeros((batch_size, num_entities))
        batch_ground_truth = np.zeros((batch_size, num_relations))
        for j, index in enumerate(indices):
            row = train_ratings.iloc[index]
            batch_relations[j, 0] = 1
            batch_entities[j, row["userId"]] = 1
            batch_entities[j, num_users + row["movieId"]] = 1
            batch_ground_truth[j, 0] = row["rating"] / 5
        # Train the model on the batch
        feed_dict = {relations: batch_relations, entities: batch_entities, ground_truth: batch_ground_truth}
        _, batch_loss = sess.run([optimizer, loss], feed_dict=feed_dict)
        # Print the training loss
        if i % 10 == 0:
            print("Epoch %d: loss = %.6f" % (i, batch_loss))

   Compute the probabilities for each movie and print out the recommended movie based on user

In [ ]:

    user_id = 0
    user_entities = np.zeros((num_users, num_entities))
    user_entities[:, user_id] = 1
    movie_entities = np.zeros((num_movies, num_entities))
    movie_entities[:, num_users:] = np.eye(num_movies)
    user_relations = np.zeros((num_users, num_relations))
    user_relations[:, 0] = 1
    feed_dict = {relations: user_relations, entities: np.concatenate([user_entities, movie_entities], axis=0)}
    user_probabilities = sess.run(probabilities, feed_dict=feed_dict)[0, num_users:]
    top_indices = np.argsort(user_probabilities)[::-1][:10]
    top_movies = [movies_df[movies_df["movieId"] == movie_id_map[index]].iloc[0]["title"] for index in top_indices]
    

In [ ]:
print("Top recommended movies for user %d:" % user_id)
    for i, movie in enumerate(top_movies):
        print("%d. %s" % (i + 1, movie)))